##### College of Engineering, Construction and Living Sciences<br>Bachelor of Information Technology<br>IN710: Object-Oriented Systems Development<br>Level 7, Credits 15<br><br>Deadline: Tuesday, 21 April at 5pm (second week of mid-semester break)

# Practical 16: SQLAlchemy

In this practical, you will complete a series of tasks covering today's lecture. This practical is worth 1% of the final mark for the Object-Oriented Systems Development course.

In [ ]:
%config IPCompleter.greedy=True

## SQLAlchemy

In [ ]:
pip install SQLAlchemy

In [ ]:
pip install PyMySQL

In [ ]:
pip install sqlalchemy-utils

### Student Courses
**Task 1:** Consider the following models & data. 

**Note:** Only run this cell once.

In [ ]:
from sqlalchemy import create_engine, Column, ForeignKey, Integer, String, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

Base = declarative_base()


class Person(Base):
    __tablename__ = 'person'
    id = Column(Integer, primary_key=True)
    first_name = Column(String(255), nullable=False)
    last_name = Column(String(255), nullable=False)
    address = Column(String(255), nullable=False)
    phone_num = Column(String(255), nullable=False)

    def __init__(self, first_name, last_name, address, phone_num):
        self.first_name = first_name
        self.last_name = last_name
        self.address = address
        self.phone_num = phone_num


class Lecturer(Person):
    __tablename__ = 'lecturer'
    id = Column(None, ForeignKey('person.id'), primary_key=True)
    courses = relationship('Course')


class Student(Person):
    __tablename__ = 'student'
    id = Column(None, ForeignKey('person.id'), primary_key=True)
    student_id = Column(Integer, nullable=False, unique=True)
    courses = relationship('Course', secondary='student_course')

    def __init__(self, first_name, last_name, address, phone_num, student_id):
        super().__init__(first_name, last_name, address, phone_num)
        self.student_id = student_id


class Course(Base):
    __tablename__ = 'course'
    id = Column(String, primary_key=True)
    lecturer_id = Column(Integer, ForeignKey('lecturer.id'))
    name = Column(String(255), nullable=False)
    semester = Column(Integer, nullable=False)
    students = relationship('Student', secondary='student_course')
    lectures = relationship('Lecture')

    def __init__(self, id, name, lecturer_id, semester):
        self.id = id
        self.name = name
        self.lecturer_id = lecturer_id
        self.semester = semester


class StudentCourse(Base):
    __tablename__ = 'student_course'
    student_id = Column(Integer, ForeignKey(
        'student.student_id'), primary_key=True)
    course_id = Column(String, ForeignKey('course.id'), primary_key=True)


class Lecture(Base):
    __tablename__ = 'lecture'
    lecture_id = Column(Integer, primary_key=True)
    course_id = Column(String(255), ForeignKey('course.id'))
    room = Column(String(255), nullable=False)

    def __init__(self, course_id, room):
        self.course_id = course_id
        self.room = room


engine = create_engine('sqlite:///teaching.db', echo=False)
Session = sessionmaker()
Session.configure(bind=engine)
Base.metadata.create_all(engine)
session = Session()

records = [
    Student('Salvador', 'Allen', '235 Ted Gilberd Place',
            '(026) 5125-882', 1000045392),
    Student('James', 'Gillespie', '94 Moray Crescent',
            '(021) 1564-974', 1000054576),
    Student('Jack', 'Carney', '47 Waimarei Avenue',
            '(022) 2549-800', 1000053613),
    Lecturer('Grayson', 'Orr', '172 Bethunes Lane', '(029) 3222-800'),
    Lecturer('Adon', 'Moskal', '260 Eclipse Terrace', '(027) 8864-363'),
    Lecturer('Michael', 'Holtz', '227 Kent Street', '(022) 7323-987'),
    Course('IN512', 'Fundamentals of Web Development', 5, 1),
    Course('IN515', 'Introduction to Networks', 6, 2),
    Course('IN615', 'Routing & Switching', 6, 1),
    Course('IN628', 'Programming 4', 4, 2),
    Lecture('IN512', 'D207'),
    Lecture('IN515', 'D313'),
    Lecture('IN615', 'D201'),
    Lecture('IN628', 'D105a'),
    StudentCourse(student_id=1000045392, course_id='IN615'),
    StudentCourse(student_id=1000045392, course_id='IN628'),
    StudentCourse(student_id=1000054576, course_id='IN512'),
    StudentCourse(student_id=1000053613, course_id='IN512'),
    StudentCourse(student_id=1000045392, course_id='IN515')
]

session.add_all(records)
session.commit()
session.close()

**Query the teaching database for the following information:**
1. All students enrolled in **IN512 Fundamentals of Web Development**. Include the student's first/last name.
2. All courses & lecturer assigned to those courses. Include the course name & lecturer's first/last name.
3. All semester two courses & rooms for **Salvador Allen**.

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()


def main():
    engine = create_engine('sqlite:///teaching.db', echo=False)
    Session = sessionmaker()
    Session.configure(bind=engine)
    Base.metadata.create_all(engine)
    session = Session()

    # Write your query one solution here

    # Write your query two solution here

    # Write your query three solution here

    session.close()


if __name__ == '__main__':
    main()

# Expected output:

# Students enrolled in IN512 Fundamentals of Web Development
# - James Gillespie
# - Jack Carney

# Courses and assigned lecturers
# - Fundamentals of Web Development - Adon Moskal
# - Introduction to Networks - Michael Holtz
# - Routing & Switching - Michael Holtz
# - Programming 4 - Grayson Orr

# Semester two timetable for Salvador Allen
# - Introduction to Networks - D313
# - Programming 4 - D105a

### Blackjack - MariaDB
**Task 2 - Research:** Consider the following model & data. This database has one table called `statistics`. This table stores a blackjack game's information such as the house's score, player's score & result. The result is represent with either a 0 (lose), 1 (win) or 2 (draw). You will be using your blackjack implementation from either practical 03 or 05.

Here are the following research task:

- Connect to mariadb.ict.op.ac.nz. The credentials are:
    - Username: `in710shared` & password: `P@ssw0rd`
- Create a database called `in710shared_<your_op_username>`
- At the start of each game, query the `statistics` table & display the current wins, loses & draws
- At the end of each game, insert the appropriate data into the `statistics` table
- Use the two helper functions below to insert rows into the `statistics` table & close the connection. **Note:** these two helper functions are not assigned to any particular class

```python
def insert_statement(session, data):
    session.add(data)
    return session.commit()

def close_connection(session):
    return session.close()
```

In [ ]:
from sqlalchemy import create_engine, Column, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import database_exists, create_database

ENGINE_URL =  # Connect to mariadb.ict.op.ac.nz

Base = declarative_base()


class Statistics(Base):
    __tablename__ = 'statistics'
    id = Column(Integer, primary_key=True)
    house_score = Column(Integer)
    player_score = Column(Integer)
    result = Column(Integer)

    def __init__(self, house_score, player_score, result):
        self.house_score = house_score
        self.player_score = player_score
        self.result = result


engine = create_engine(ENGINE_URL, echo=False)
# Create database if it doesn't exist
if not database_exists(engine.url):
    create_database(engine.url)

Session = sessionmaker()
Session.configure(bind=engine)
Base.metadata.create_all(engine)
session = Session()
session.add_all([
    Statistics(21, 18, 0),
    Statistics(18, 21, 1),
    Statistics(18, 18, 2)
])
session.commit()
session.close()

In [ ]:
# Write your solution here

# Submission
1. Create a new branch named 16-checkpoint within your practicals GitHub repository
2. Create a new pull request and assign Grayson-Orr to review your submission

**Note:** Please don't merge your own pull request.